In [1]:
import requests
import json
import datetime as dt
import pandas as pd
import yfinance as yf
import numpy as np
import pandas_ta as ta
import pyupbit

In [2]:
def get_data(symbol,interval,start):    
    url = 'https://api.binance.com/api/v3/klines'
    start1 = str(int(start.timestamp()*1000))
    end = str(int((start+dt.timedelta(days = 1)).timestamp()*1000))
    start = start1
    par = {'symbol': symbol, 'interval': interval, 'startTime': start, 'endTime': end}
    data = pd.DataFrame(json.loads(requests.get(url, params= par).text))
    data.columns = ['datetime', 'Open', 'High', 'Low', 'Close', 'volume','close_time', 'qav', 'num_trades','taker_base_vol', 'taker_quote_vol', 'ignore']
    data.index = [dt.datetime.fromtimestamp(x/1000.0) for x in data.datetime]
    data=data.astype(float)
    df = data.iloc[:,1:5]
    df.index.name = "Date Time"
    return df

In [3]:
def get_data_duration(symbol,interval,start,duration):
    start = dt.datetime(start[0],start[1],start[2]) - dt.timedelta(days = duration)
    data = get_data(symbol,interval,start)
    for _ in range(duration-1):
        start = start+dt.timedelta(days = 1)
        temp = get_data(symbol,interval,start)
        if interval != "1d":
            data = pd.concat([data,temp[1:]])
        else:
            data = pd.concat([data,temp])
    return data

In [4]:
def rsi_bb(symbol,interval,start,duration,standard):
    df = get_data_duration(symbol,interval,start,duration)
    df["Rsi"] = ta.rsi(close = df.Close,length = 14) 
    bb = ta.bbands(close = df['Close'],length = 20,std = standard).iloc[:,0:3]
    df["BB_LOW"] = bb.iloc[:,0]
    df["BB_MID"] = bb.iloc[:,1]
    df["BB_HIGH"] = bb.iloc[:,2]
    df = df.dropna()
    return df

In [5]:
def get_data_duration2(symbol,interval,start,duration):
    start = dt.datetime(start[0],start[1],start[2])
    data = get_data(symbol,interval,start)
    for _ in range(duration-1):
        start = start+dt.timedelta(days = 1)
        temp = get_data(symbol,interval,start)
        if interval != "1d":
            data = pd.concat([data,temp[1:]])
        else:
            data = pd.concat([data,temp])
    return data

In [6]:
def get_upbit_binance(upticker,upinterval,biticker,biinterval,date,duration):
    df1 = pyupbit.get_ohlcv(ticker=upticker,interval=upinterval,to=f"{date[0]}-{date[1]}-{date[2]}",count=duration).iloc[:,0:4]
    df1=df1.rename(columns={'open':'Open','high':'High','low':'Low','close':'Close'})
    df2 = get_data_duration(biticker,biinterval,date,duration)
    return df1,df2
    

In [7]:
def get_5min_data(upticker,biticker,date):
    a = pyupbit.get_ohlcv(ticker = upticker,interval = "minute5",to = f"{date[0]}-{date[1]}-{date[2]} 15:05",count = 289)
    b = get_data_duration2(biticker,"5m",date,1)
    df=pd.merge(a["close"], b["Close"], left_index=True, right_index=True)
    df["usd/krw"] = df["Close"].div(df["close"])*1304
    df = df[:-1]
    df=df.rename(columns={'close':'upclose','Close':'biclose'})
    return df

In [8]:
def rebalance(usd,krw,ku):
    temp = usd*ku + krw
    krw = temp/2
    usd = krw/ku
    return usd,krw
    

In [25]:
def kimp(start,duration,uptick,bitick):
    start_date = dt.datetime(start[0],start[1],start[2])
    end_date = start_date + dt.timedelta(days = duration+6)
    df1,df2 = get_upbit_binance(uptick,"day1",bitick,"1d",(end_date.year,end_date.month,end_date.day),duration+12)
    df3 = (df2["Close"].div(df1["Close"])*1304).to_frame()
    krw_usd = yf.download("KRW=X",start = f"{start_date.year}-{start_date.month}-{start_date.day}",end = f"{end_date.year}-{end_date.month}-{end_date.day}")
    krw_usd = krw_usd["Close"]
    usd = 1000
    krw = 1000*krw_usd[start_date]
    start_bal = 2*krw
    forex = krw_usd[start_date]
    flag = True
    history = []
    
    
    
    for j in range(duration):
        cur_date = start_date + dt.timedelta(days = j)
        cur = (cur_date.year,cur_date.month,cur_date.day)
        df = get_5min_data(uptick,bitick,cur)
        past4 = [cur_date-dt.timedelta(days=5),cur_date-dt.timedelta(days=4),cur_date-dt.timedelta(days=3),cur_date-dt.timedelta(days=2)]
        past4 = [df3.loc[past4[0].strftime('%Y-%m-%d'),"Close"].item(),df3.loc[past4[1].strftime('%Y-%m-%d'),"Close"].item(),df3.loc[past4[2].strftime('%Y-%m-%d'),"Close"].item(),df3.loc[past4[3].strftime('%Y-%m-%d'),"Close"].item()]
        for i in df.index:
            cur_price = df.loc[i,"usd/krw"].item()
            mean = (past4[0]+past4[1]+past4[2]+past4[3]+cur_price)/5
            std = np.std([past4[0],past4[1],past4[2],past4[3],cur_price])
            bh = mean+std*1.3
            bl = mean-std*1.3
            if not bh>cur_price>bl:
                if bl>cur_price and flag:
                    history.append([i,df.loc[i,"upclose"].item(),df.loc[i,"biclose"].item()])
                    flag = not flag
                elif bh<cur_price and not flag:
                    history.append([i,df.loc[i,"upclose"].item(),df.loc[i,"biclose"].item()])
                    flag = not flag
    sum = 0
    bal = 100
    for i,arr in enumerate(history[:-1]):
        pc = (history[i+1][1]-history[i][1])/history[i][1] - (history[i+1][2]-history[i][2])/history[i][2]
        if i%2 ==0:
            pc = -pc
        print(history[i][0],history[i+1][0],pc*100)
        print(history[i][1],history[i][2],history[i+1][1],history[i+1][2])
        sum+=pc*100 - 0.20
        bal*=(pc-0.0025)/2+1
    if cur_date not in krw_usd.index:
        cur_date = cur_date - dt.timedelta(days=2)
    print(f"sum = {sum}",forex,krw_usd[cur_date])
    print(bal)
            
            
            
              
    

In [32]:
kimp((2022,6,15),37,"KRW-TRX","TRXBUSD")

[*********************100%***********************]  1 of 1 completed
2022-06-15 16:55:00 2022-06-16 00:50:00 1.2398717613623267
63.1 0.0479 72.0 0.05525
2022-06-16 00:50:00 2022-06-16 01:50:00 0.5717697335344496
72.0 0.05525 70.9 0.05409
2022-06-16 01:50:00 2022-06-16 04:05:00 0.6998209378351758
70.9 0.05409 77.6 0.05958
2022-06-16 04:05:00 2022-06-16 18:05:00 0.6855548980319368
77.6 0.05958 79.2 0.0604
2022-06-16 18:05:00 2022-06-20 05:30:00 1.4194929426717446
79.2 0.0604 80.2 0.06202
2022-06-20 05:30:00 2022-06-29 03:10:00 1.4219530181318663
80.2 0.06202 86.5 0.06601
2022-06-29 03:10:00 2022-07-08 00:55:00 0.41794333140976575
86.5 0.06601 89.1 0.06827
2022-07-08 00:55:00 2022-07-11 00:20:00 0.9178581709219884
89.1 0.06827 89.2 0.06772
2022-07-11 00:20:00 2022-07-18 14:15:00 0.3447325971621569
89.2 0.06772 91.0 0.06932
2022-07-18 14:15:00 2022-07-21 02:10:00 0.8952588092807523
91.0 0.06932 90.2 0.06809
2022-07-21 02:10:00 2022-07-21 08:00:00 0.8831567974954333
90.2 0.06809 88.9 0.0677